In [39]:

from sqlalchemy.orm import sessionmaker
from models import connect_db, PointsOfInterest, ArchitecturalStyles, Architects,POICategories
import pandas as pd
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder
import os
from dotenv import load_dotenv, find_dotenv
import time
DEBUG=0

In [40]:
def get_bing_key():
    load_dotenv(find_dotenv())
    # load environment variables
    BING_KEY = os.getenv("BING_KEY")
    DEBUG = os.getenv("DEBUG")
    return BING_KEY

In [47]:
def get_lats_longs(address):
    '''This function uses GeoCoder to retrieve the latitude and longitude for neighbourhoods.
    address to look up
    returns lat and long
    '''
    BING_KEY = get_bing_key()
    # # get geocode for postal codes
    # initialize your variable to None
    lat_lng_coords = None
    lat=None
    long=None
    max_loops = 10
    curr_loop =0
    # loop until you get the coordinates (sometime dont' get a response)
    lat_lng_coords = None
    while(lat_lng_coords is None and curr_loop < max_loops):
        g = geocoder.bing('{}'.format(address),key=BING_KEY)
        if DEBUG > 0:
            print (g)
        lat_lng_coords = g.latlng
        if DEBUG > 0:
            print(lat_lng_coords)
        if lat_lng_coords == None:
            time.sleep(1)
        curr_loop += 1
    if lat_lng_coords is not None:
        lat=lat_lng_coords[0]
        long=lat_lng_coords[1]

    return lat,long

In [48]:
def update_missing_coords():
    '''
    Find all rows in PointsofINterest table that are missing lat and long
    Lookup address using Geocoder and update database row
    '''
    null_lats = session.query(PointsOfInterest).filter(PointsOfInterest.latitude == None)
    for row in null_lats:
        #null_lat = session.query(PointsOfInterest).get(1)
        lat, long= get_lats_longs(row.address)
        props = {'latitude':lat, 'longitude':long}

        for key, value in props.items():
            setattr(row, key, value)

    # save up our commits
    session.commit()
    session.flush()
    return True


In [53]:
db=connect_db() #establish connection / creates database on first run
Session = sessionmaker(bind=db)
session = Session()
num_missing_coords = session.query(PointsOfInterest).filter(PointsOfInterest.latitude == None).count()
if num_missing_coords> 0:
    update_missing_coords()

In [54]:
#session.query(PointsOfInterest).get(3500).latitude #43.66449
num_missing_coords = session.query(PointsOfInterest).filter(PointsOfInterest.latitude == None).count()
num_missing_coords

35

In [55]:
session.commit()

In [22]:
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import numpy as np

In [15]:
df = pd.read_sql(session.query(PointsOfInterest).statement, session.bind)
df.head()

,poi_id,name,build_year,demolished_year,address,latitude,longitude,external_url,image_url,heritage_status,current_use,poi_type,source,details
0,3224,196-198 First Avenue,1892,None,196-198 First Avenue Riverdale Toronto,43.66641,-79.34533,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,Riverdale Heritage Conservation District,Residential,Semi-detached house,None,The original owner of this building was George...
1,3225,200-202 First Avenue,1892,None,200-202 First Avenue Riverdale Toronto,43.66644,-79.34518,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,Riverdale Heritage Conservation District,Residential,Semi-detached house,None,The original owner of this building was George...
2,3226,204-206 First Avenue,1889,None,204-206 First Avenue Riverdale Toronto,43.66647,-79.34507,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,Riverdale Heritage Conservation District,Residential,Semi-detached house,None,The original owners of this building was Alfre...
3,3227,76 Gerrard Street East,1879,None,76 Gerrard Street East Church-Wellesley Toronto,43.66012,-79.37813,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,Heritage property,Residential,Detached house,None,None
4,3228,William Muir House (Glen Road),1884,None,2 Glen Road St. Jamestown Toronto,43.67146,-79.37515,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,Designated under Part IV Ontario Heritage Act,Residential,Semi-detached house,None,Representative of Toronto Bay-n-Gable architec...


In [20]:
map_clusters = folium.Map(location=[43.67146, -79.37515], zoom_start=11)

mc = MarkerCluster()
#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in df.iloc[1:10,:].itertuples():
    mc.add_child(folium.CircleMarker(location=[row.latitude,  row.longitude]))

map_clusters.add_child(mc)
map_clusters

In [25]:
map_clusters = folium.Map(location=[43.67146, -79.37515], zoom_start=11)
# set color scheme for the clusters
# x = np.arange(len(set_clusters))
# ys = [i+x+(i*x)**2 for i in range(len(set_clusters))]
# colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
# rainbow = [colors.rgb2hex(i) for i in colors_array]

# callback = ('function (row) {'
#              'var circle = L.circle(new L.LatLng(row[0], row[1],' 
#             '{color: "red", radius: 5});'
#              'return circle};')
# callback = """\
# function (row) {
#     var icon, marker;
#     icon = L.AwesomeMarkers.icon({
#         icon: "map-marker", markerColor: "red"});
#     marker = L.marker(new L.LatLng(row[0], row[1]));
#     marker.setIcon(icon);
#     return marker;
# };
# """
callback = """\
function (row) {
    var icon, marker;
    var circle = L.circle(new L.LatLng(row[0],row[1],
    {color:'blue', radius:5}
    ))
    
    return circle;
};
"""

map_clusters.add_child(FastMarkerCluster(df[['latitude','longitude']].values.tolist(), callback=callback))


map_clusters

In [ ]:
 icon = L.AwesomeMarkers.icon({markerColor: row.color})    
    marker = L.marker(L.LatLng(row.lat, row.lng))
    marker.setIcon(icon)
    if popup:
        marker.bindPopup(row[popup])

In [38]:
map_clusters = folium.Map(location=[43.67146, -79.37515], zoom_start=11)

callback = """\
function (row) {
    var icon, marker;
     var circle = L.circle(new L.LatLng(row[0],row[1],
        {color:'blue', radius:5}
        ))
    circle.bindPopup('test')
    return circle;
};
"""

map_clusters.add_child(FastMarkerCluster(df[['latitude','longitude']].values.tolist(), callback=callback))


map_clusters

#EDA todod
* group by age of building
* group by neighbourhood
* 